In [ ]:
import os
import time
import utils
import math
import argparse
import datetime
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.distributed.optim
import torch.utils.data
from torch.utils.tensorboard import SummaryWriter

import torchvision
from spikingjelly.activation_based import layer,functional,neuron,surrogate


In [2]:
torch.backends.cudnn.benchmark = True
np.int = int

train_tb_writer = None
te_tb_writer = None
device = torch.device('cuda:0')
deviceIds = [0]

In [3]:
def parse_args():
    parser = argparse.ArgumentParser(description='PyTorch Classification Training')

    parser.add_argument('--device', default='cuda:0', help='device')
    parser.add_argument('-b', '--batch-size', default=32, type=int)
    parser.add_argument('--data-path', default='./data/', help='dataset')
    parser.add_argument('--epochs', default=320, type=int, metavar='N',
                        help='number of total epochs to pre-train')
    parser.add_argument('-j', '--workers', default=16, type=int, metavar='N',
                        help='number of data loading workers (default: 16)')
    parser.add_argument('--lr', default=0.0025, type=float, help='initial learning rate')

    parser.add_argument('--print-freq', default=10, type=int, help='print frequency')
    parser.add_argument('--output-dir', default='./logs', help='path where to save')
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--tb', action='store_true',
                        help='Use TensorBoard to record logs')
    parser.add_argument(
        "--cache-dataset",
        dest="cache_dataset",
        help="Cache the datasets for quicker initialization. It also serializes the transforms",
        action="store_true",
    )

    args = parser.parse_args(args=['--data-path','../data','--lr','0.01','-b','128','--epochs','100','--print-freq','100','--tb','--cache-dataset'])
    return args

args = parse_args()


In [ ]:
class NetworkB(torch.nn.Module):
    def __init__(self):
        super(NetworkB, self).__init__()
        self.T = 10
        self.conv1 = nn.Conv2d(1, 96, kernel_size=3, padding=3) 
        self.bn1 = nn.BatchNorm2d(96)
        self.sn1 = neuron.IFNode(detach_reset=True)

        self.conv2 = layer.SeqToANNContainer(nn.Conv2d(96, 96, kernel_size=3, padding=1),nn.BatchNorm2d(96)) 
        self.sn2 = neuron.IFNode(detach_reset=True)
        self.pool1 = layer.SeqToANNContainer(nn.MaxPool2d(2)) 

        self.conv3 = layer.SeqToANNContainer(nn.Conv2d(96, 128, kernel_size=3, padding=1),nn.BatchNorm2d(128)) 
        self.sn3 = neuron.IFNode(detach_reset=True)
      
        self.conv4 = layer.SeqToANNContainer(nn.Conv2d(128, 128, kernel_size=3, padding=1),nn.BatchNorm2d(128)) 
        self.sn4 = neuron.IFNode(detach_reset=True)
        self.pool2 = layer.SeqToANNContainer(nn.MaxPool2d(2)) 

        self.conv5 = layer.SeqToANNContainer(nn.Conv2d(128, 256, kernel_size=3, padding=1),nn.BatchNorm2d(256)) 
        self.sn5 = neuron.IFNode(detach_reset=True)


        self.pool3 = layer.SeqToANNContainer(nn.MaxPool2d(2)) 

        self.conv8 = layer.SeqToANNContainer(nn.Conv2d(256, 512, kernel_size=3, padding=1),nn.BatchNorm2d(512)) 
        self.sn8 = neuron.IFNode(detach_reset=True)

        self.conv10 = layer.SeqToANNContainer(nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.BatchNorm2d(512)) 
        self.sn10 = neuron.IFNode(detach_reset=True)

        self.pool4 = layer.SeqToANNContainer(nn.MaxPool2d(2)) 

        self.conv11 = layer.SeqToANNContainer(nn.Conv2d(512, 256, kernel_size=3, padding=1),nn.BatchNorm2d(256)) 
        self.sn11 = neuron.IFNode(detach_reset=True)

        self.conv13 = layer.SeqToANNContainer(nn.Conv2d(256, 128, kernel_size=3, padding=1),nn.BatchNorm2d(128)) 
        self.sn13 = neuron.IFNode(detach_reset=True)

        self.fc1 = layer.SeqToANNContainer(nn.Linear(512, 256),nn.BatchNorm1d(256))
        self.sn14 = neuron.IFNode(detach_reset=True)
        self.fc2 = layer.SeqToANNContainer(nn.Linear(256, 10),nn.BatchNorm1d(10))
        self.sn15 = neuron.IFNode(detach_reset=True)
        

    def forward(self, x):
        T = self.T
        x = self.bn1(self.conv1(x))
        x.unsqueeze_(0)
        x = x.repeat(10, 1, 1, 1, 1)
        x = self.sn1(x)

        x = self.sn2(self.conv2(x))
        x = self.pool1(x)

        x = self.sn3(self.conv3(x))
        x = self.sn4(self.conv4(x))
        x = self.pool2(x)

        x = self.sn5(self.conv5(x))
        x = self.pool3(x)

        x = self.sn8(self.conv8(x))
        x = self.sn10(self.conv10(x))
        x = self.pool4(x)

        x = self.sn11(self.conv11(x))

        x = x[:T, :, :, :, :]
       
        x = self.sn13(self.conv13(x))
      
        x = torch.flatten(x,2)

        x = self.sn14(self.fc1(x))
        x = self.sn15(self.fc2(x))
       
        return x.mean(0)
    
    def set_T(self, T):
        self.T = T

In [ ]:
args = parse_args()
max_test_acc1 = 0.
test_acc5_at_max_test_acc1 = 0.

utils.init_distributed_mode(args)
print(args)
output_dir = os.path.join(args.output_dir, f'b_{args.batch_size}_lr{args.lr}')

time_now = datetime.datetime.now()
output_dir += f'_{time_now.year}_{time_now.month}_{time_now.day}_{time_now.hour}__{time_now.minute}'


if output_dir:
    utils.mkdir(output_dir)

train_dataset = torchvision.datasets.MNIST(
    root=args.data_path,
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root=args.data_path,
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

data_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=args.workers,
    pin_memory=True
)
data_loader_test = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=args.workers,
    pin_memory=True
)

print("Creating model")

net = NetworkB().to(device)
functional.set_step_mode(net,step_mode='m')
functional.set_backend(net, backend='cupy')
net.to(device)

optimizer = torch.optim.SGD(
    net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.4, last_epoch=-1)

criterion = nn.CrossEntropyLoss()

if args.resume:
    checkpoint = torch.load(args.resume, map_location='cpu')
    args.start_epoch = checkpoint['epoch'] + 1
    max_test_acc1 = checkpoint['max_test_acc1']
    test_acc5_at_max_test_acc1 = checkpoint['test_acc5_at_max_test_acc1']


if args.tb and utils.is_main_process():
    purge_step_train = args.start_epoch
    purge_step_te = args.start_epoch
    train_tb_writer = SummaryWriter(output_dir + '_logs/train', purge_step=purge_step_train)
    te_tb_writer = SummaryWriter(output_dir + '_logs/te', purge_step=purge_step_te)
    with open(output_dir + '_logs/args.txt', 'w', encoding='utf-8') as args_txt:
        args_txt.write(str(args))

    with open(output_dir + '_logs/resluts.txt', 'w', encoding='utf-8') as args_txt:
        args_txt.write('Results\n')


Not using distributed mode
Namespace(batch_size=128, cache_dataset=True, data_path='../data', device='cuda:0', distributed=False, epochs=100, lr=0.01, output_dir='./logs', print_freq=100, resume='', start_epoch=0, tb=True, workers=16)
Creating model


In [7]:
def train_one_epoch(net, criterion, data_loader, device, epoch, print_freq, scaler=None,lr = 1e-2):
    net.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value}'))
    metric_logger.add_meter('img/s', utils.SmoothedValue(window_size=10, fmt='{value}'))

    header = 'Epoch: [{}]'.format(epoch)


    for image, target in metric_logger.log_every(data_loader, print_freq, header):
        start_time = time.time()
        image, target = image.to(device), target.to(device)

        output = net(image)
        loss = criterion(output, target)

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()


        functional.reset_net(net)

        acc1, acc5 = utils.accuracy(output, target, topk=(1, 5))
        batch_size = image.shape[0]
        loss_s = loss.item()
        if math.isnan(loss_s):
            raise ValueError('loss is Nan')
        acc1_s = acc1.item()
        acc5_s = acc5.item()

        metric_logger.update(loss=loss_s, lr=lr)

        metric_logger.meters['acc1'].update(acc1_s, n=batch_size)
        metric_logger.meters['acc5'].update(acc5_s, n=batch_size)
        metric_logger.meters['img/s'].update(batch_size / (time.time() - start_time))

 
    metric_logger.synchronize_between_processes()
    return metric_logger.loss.global_avg, metric_logger.acc1.global_avg, metric_logger.acc5.global_avg

In [8]:
def evaluate(net, criterion, data_loader, device, print_freq=100, header='Test:'):
    net.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    with torch.no_grad():
        for image, target in metric_logger.log_every(data_loader, print_freq, header):
            image = image.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            output = net(image)
            loss = criterion(output, target)
            functional.reset_net(net)

            acc1, acc5 = utils.accuracy(output, target, topk=(1, 5))
           
            batch_size = image.shape[0]
            metric_logger.update(loss=loss.item())
            metric_logger.meters['acc1'].update(acc1.item(), n=batch_size)
            metric_logger.meters['acc5'].update(acc5.item(), n=batch_size)
    
    metric_logger.synchronize_between_processes()

    loss, acc1, acc5 = metric_logger.loss.global_avg, metric_logger.acc1.global_avg, metric_logger.acc5.global_avg
    print(f' * Acc@1 = {acc1}, Acc@5 = {acc5}, loss = {loss}')
    return loss, acc1, acc5

In [9]:
print("Start training")
start_time = time.time()
for epoch in range(args.start_epoch, args.epochs):
    save_max = False

    train_loss, train_acc1, train_acc5 = train_one_epoch(net, criterion,data_loader, device, epoch, args.print_freq,lr=args.lr)

    if utils.is_main_process():
        train_tb_writer.add_scalar('train_loss', train_loss, epoch)
        train_tb_writer.add_scalar('train_acc1', train_acc1, epoch)
        train_tb_writer.add_scalar('train_acc5', train_acc5, epoch)
    
    test_loss, test_acc1, test_acc5 = evaluate(net, criterion, data_loader_test, device=device, header='Test:')
    if te_tb_writer is not None:
        if utils.is_main_process():
            te_tb_writer.add_scalar('test_loss', test_loss, epoch)
            te_tb_writer.add_scalar('test_acc1', test_acc1, epoch)
            te_tb_writer.add_scalar('test_acc5', test_acc5, epoch)


    if max_test_acc1 < test_acc1:
        max_test_acc1 = test_acc1
        test_acc5_at_max_test_acc1 = test_acc5
        save_max = True

    if output_dir:

        checkpoint = {
            'model': net.state_dict(),
            'pre-train_epoch': epoch,
            'args': args,
            'max_test_acc1': max_test_acc1,
            'test_acc5_at_max_test_acc1': test_acc5_at_max_test_acc1,
        }

        utils.save_on_master(
            checkpoint,
            os.path.join(output_dir, 'checkpoint_latest.pth'))
        save_flag = False

        if epoch % 64 == 0 or epoch == args.epochs - 1:
            save_flag = True


        if save_flag:
            utils.save_on_master(
                checkpoint,
                os.path.join(output_dir, f'checkpoint_{epoch}.pth'))

        if save_max:
            utils.save_on_master(
                checkpoint,
                os.path.join(output_dir,f'train_maxacc1_{max_test_acc1}_checkpoint_max_test_acc1_all_pretrain.pth'))
            torch.save(net,os.path.join(output_dir,f'train_maxacc1_{max_test_acc1}_checkpoint_max_test_acc1_all_pretrain.pth'))
            torch.save(net.state_dict(),os.path.join(output_dir,f'train_maxacc1_{max_test_acc1}_checkpoint_max_test_acc1_state_pretrain.pth'))
    print(args)
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print(output_dir)

    print('Training time {}'.format(total_time_str), 'max_test_acc1', max_test_acc1,
            'test_acc5_at_max_test_acc1', test_acc5_at_max_test_acc1,'train_acc1', train_acc1,
            'train_acc5', train_acc1)

    Train_logs= {
        'Epoch:': epoch,
        'max_test_acc1 ': max_test_acc1 ,
        'test_acc5_at_max_test_acc1 ': test_acc5_at_max_test_acc1,
        'train_acc1 ': train_acc1,
        'train_acc5 ': train_acc1,
        'args': args
    }
    with open(output_dir + '_logs/args.txt', 'a', encoding='utf-8') as args_txt:
        args_txt.write('\n')
        args_txt.write(str(Train_logs))

Start training
Epoch: [0]  [  0/468]  eta: 0:28:10  lr: 0.01  img/s: 41.13945467537418  loss: 2.3051 (2.3051)  acc1: 6.2500 (6.2500)  acc5: 54.6875 (54.6875)  time: 3.6127  data: 0.5013  max mem: 10434
Epoch: [0]  [100/468]  eta: 0:01:16  lr: 0.01  img/s: 703.8742213895484  loss: 1.5493 (1.7661)  acc1: 92.1875 (70.4440)  acc5: 99.2188 (94.0439)  time: 0.1642  data: 0.0004  max mem: 10434
Epoch: [0]  [200/468]  eta: 0:00:49  lr: 0.01  img/s: 944.9340798406074  loss: 1.5008 (1.6426)  acc1: 96.0938 (82.7309)  acc5: 99.2188 (96.7390)  time: 0.1236  data: 0.0004  max mem: 10434
Epoch: [0]  [300/468]  eta: 0:00:30  lr: 0.01  img/s: 705.1472522853841  loss: 1.4937 (1.5936)  acc1: 96.8750 (87.4143)  acc5: 99.2188 (97.6900)  time: 0.1759  data: 0.0004  max mem: 10434
Epoch: [0]  [400/468]  eta: 0:00:12  lr: 0.01  img/s: 701.7324175070092  loss: 1.4906 (1.5671)  acc1: 97.6562 (89.9509)  acc5: 100.0000 (98.2135)  time: 0.1751  data: 0.0004  max mem: 10434
Epoch: [0] Total time: 0:01:23
Test:  [ 0

In [ ]:
model_dir = os.path.join(output_dir,f'train_maxacc1_{max_test_acc1}_checkpoint_max_test_acc1_state_pretrain.pth')

In [11]:
acc_list = []
for time in range(1,11):
    acc = 0
    for i in range(10):
        net = NetworkB().to(device)
        functional.reset_net(net)
        functional.set_step_mode(net,step_mode='m')
        functional.set_backend(net, backend='cupy')
        weights = torch.load(model_dir)
        net.load_state_dict(weights)
        net.set_T(time)
        test_loss, test_acc1, test_acc5 = evaluate(net, criterion, data_loader_test, device=device, header='Test:')
        acc += test_acc1
    acc_list.append(acc/10)

Test:  [ 0/79]  eta: 0:01:02  loss: 1.4807 (1.4807)  acc1: 97.6562 (97.6562)  acc5: 99.2188 (99.2188)  time: 0.7888  data: 0.3548  max mem: 10434
Test: Total time: 0:00:05
 * Acc@1 = 98.46, Acc@5 = 99.2, loss = 1.4743724261658102
Test:  [ 0/79]  eta: 0:00:34  loss: 1.4807 (1.4807)  acc1: 97.6562 (97.6562)  acc5: 99.2188 (99.2188)  time: 0.4317  data: 0.3608  max mem: 10434
Test: Total time: 0:00:05
 * Acc@1 = 98.46, Acc@5 = 99.2, loss = 1.4743724261658102
Test:  [ 0/79]  eta: 0:00:32  loss: 1.4807 (1.4807)  acc1: 97.6562 (97.6562)  acc5: 99.2188 (99.2188)  time: 0.4145  data: 0.3489  max mem: 10434
Test: Total time: 0:00:05
 * Acc@1 = 98.46, Acc@5 = 99.2, loss = 1.4743724261658102
Test:  [ 0/79]  eta: 0:00:35  loss: 1.4807 (1.4807)  acc1: 97.6562 (97.6562)  acc5: 99.2188 (99.2188)  time: 0.4481  data: 0.3806  max mem: 10434
Test: Total time: 0:00:05
 * Acc@1 = 98.46, Acc@5 = 99.2, loss = 1.4743724261658102
Test:  [ 0/79]  eta: 0:00:43  loss: 1.4807 (1.4807)  acc1: 97.6562 (97.6562)  ac

In [ ]:
with open('output_snn.txt', 'a') as file:
    line = ' '.join(map(str, acc_list))
    file.write(line + '\n')